In [6]:
import numpy as np

from src.a_function import make_labels
from src.a_variable import train_y

imgs = np.load('./save/1-stage-myimg-train-384.npy')
labels, label_unique = make_labels(train_y, target='label')

In [1]:
# from sklearn.model_selection import train_test_split

# train_img, val_img, train_label, val_label = train_test_split(
#     imgs, labels, test_size=0.2, stratify=labels
# )
# train_ratio = Counter(train_label)
# over_label = [key for key in train_ratio.keys() if train_ratio[key]>100]
# over_set = [(img, label) for img, label in zip(train_img, train_label) if label in over_label]
# over_img = [x[0] for x in over_set]
# over_label = [x[1] for x in over_set]
# print(len(train_img), len(over_img), len(over_label))

# val_ratio = Counter(val_label)
# over_label = [val_ratio[key] for key in val_ratio.keys()]
# sorted(over_label, reverse=True)
# # over_set = [(img, label) for img, label in zip(train_img, train_label) if label in over_label]
# # over_img = [x[0] for x in over_set]
# # over_label = [x[1] for x in over_set]
# # print(len(train_img), len(over_img), len(over_label))
# # [(label, train_ratio[label]) for img, label in zip(train_img, train_label) if label in over_label]

In [2]:
# print(len(train_label), len(val_label))

# from collections import Counter

# train_ratio = Counter(train_label)
# val_ratio = Counter(val_label)

# print('Labels:', len(train_ratio.keys()), len(val_ratio.keys()))
# for train_key, val_key in zip(sorted(train_ratio.keys()), sorted(val_ratio.keys())):
#     print(f'Train - {train_key}: {train_ratio[train_key]}, Val - {val_key}: {val_ratio[val_key]}')

In [8]:
import torch

from src.a_variable import device
from src.a_class import Network
from src.a_compilation import make_data, make_test_loader, make_submission

stage = '1-stage'
size = 384

lr = 2e-4
wd = 2e-2

batch_size = 8

aug_on = True
aug_state = 'On' if aug_on else 'Off'

under_multiple=2

train_imgs, train_labels, label_unique, test_imgs = make_data(stage, size)
test_loader = make_test_loader(test_imgs, batch_size)

model_test = Network(len(label_unique), mode='test').to(device)
model_test.load_state_dict(torch.load(('./save/best_model_2.pth'))['state_dict'])
model_test.eval()
pred_prob = []
with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model_test(x)
        pred = pred.argmax(1).detach().cpu().numpy().tolist()
        pred_prob.extend(pred)

make_submission(label_unique, pred_prob,
                stage, size, aug_state, under_multiple)

C:\Users\손재원\AppData\Local\Temp/ipykernel_17488/3984189453.py:29: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(batch[0], dtype = torch.float32, device = device)


In [17]:
import numpy as np
xxx = np.load('./save/1-stage-myimg-test-384.npy')
[np.mean(x, axis=(0,1)) for x in xxx[:2]]

[array([98.2954305 , 99.63859728, 95.27581109]),
 array([92.71169027, 92.71169027, 92.71169027])]